Esse arquivo não cria o banco de dados no postgre, ele assume que o banco foi criado a partir do PGAdmin, e foi utilizado tambem o arquivo ou modelo.sql para criar todas as tabelas, ou foi utilizado o arquivo modelo.pgrd para criar as tabelas. Ambas formas, devem dar o mesmo resultado.
O objetivo deste notebook é inserir todos os dados do csv no banco do postgre.

Na célula abaixo foi importadas as bibliotecas que serão utilizadas.
Para rodar esse notebook, não se esqueça de criar um ambiente virtual e instalar a dependencias que estão em requirements.txt
Pode utilizar pip install requirements.txt

In [ ]:
import pandas as pd, numbers
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, text
from sqlalchemy.orm import relationship, Session, declarative_base
from sqlalchemy.ext.automap import automap_base



Aqui foi inicializado a conexao com o banco postgre em seu computador, por meio de uma conexao localhost.
E tambem foram importadas o modelo das tabelas para que possa ser feita a inserção por meio do SQL_Alchemy

In [ ]:

username = "postgres"
senha = "suasenha" # aqui coloque sua senha do servidor postgre
host= "127.0.0.1"
nome_banco = "projeto1v1"

string_conecta = f"postgresql://{username}:{senha}@{host}/{nome_banco}"

engine = create_engine(string_conecta)

Base = automap_base()
Base.prepare(engine, reflect=True)

Food = Base.classes.Food
Animals = Base.classes.Animals
Stocks = Base.classes.Stocks
Production = Base.classes.Production
Area_harvest = Base.classes.Area_harvest
Area = Base.classes.Area

session = Session(engine)

/tmp/ipykernel_14720/2349054361.py:12: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  Base.prepare(engine, reflect=True)


Lê os dados das comidas a partir do arquivo csv para inserir na tabela Food

In [12]:
input_path_Food = "./crops_animals/crops_animals.csv"

cols = ['Area Code','Area Code (M49)','Area','Item Code','Item Code (CPC)','Item','Element Code','Element','Year Code','Year','Unit','Value','Flag','Note']

reader = pd.read_csv(input_path_Food,
                     delimiter=',')

data = []
counter = 0

map = {}

for _,item in reader.iterrows():
    counter = counter + 1
    obj_food = {'id_food':counter, 'code':item['Item Code'], 'code(cpc)':item['Item Code (CPC)'],'name':item['Item']}
    
    if(map.get(obj_food['name']) is None):
        data.append(obj_food)
        map[obj_food['name']] = counter




/tmp/ipykernel_14720/2918162328.py:5: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  reader = pd.read_csv(input_path_Food,


Realiza uma chamada no postgre atraves do bulk_insert para inserir todos os dados de uma vez, de acordo com a documentacao é mais eficiente do que chamar uma linha de insert por dado

In [13]:
Food = Base.classes.Food
try:
    session.bulk_insert_mappings(Food,data)
    session.commit()
except Exception as e:
    print("Erro ao fazer bulk insert mappings : ", e)
    session.rollback()


Lê os dados dos aniamis a partir do arquivo csv para inserir na tabela Animals

In [ ]:
input_path = './data/crops_animals/crops_animals.csv'
cols = ['Area Code','Area Code (M49)','Area','Item Code','Item Code (CPC)','Item','Element Code','Element','Year Code','Year','Unit','Value','Flag','Note']
reader = pd.read_csv(input_path, 
                    delimiter=',')

data_animals = []
counter_animals = 0
map_animals = {}

for _,item in reader.iterrows():

    if item['Element Code'] == 5111 or item['Element Code'] == 5112 or item['Element Code'] == 5114:
        obj_animals = {'id_animals':counter_animals, 'code':item['Item Code'], 'code(cpc)':item['Item Code (CPC)'], 'name':item['Item']}
        
        if(map_animals.get(obj_animals['name']) == None):
            counter_animals = counter_animals + 1

            if item['Unit'] == None or  isinstance(item['Unit'], numbers.Number):
                obj_animals['unit'] = 0

            data_animals.append(obj_animals)
            map_animals[obj_animals['name']] = obj_animals['id_animals']

                


/tmp/ipykernel_14720/1026636154.py:3: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  reader = pd.read_csv(input_path,


Realiza chamada no postgre para inserir os animais

In [15]:
try:
    session.bulk_insert_mappings(Animals,data_animals)
    session.commit()
    data_animals = []
except Exception as e:
    print("Erro ao fazer bulk insert mappings dos Animals : ", e)
    session.rollback()


Lê os dados das Areas dos 3 arquivos csv, é a tabela com as informações que mais se cruzam entre todas as tabelas, por isso há um tratamento maior dos dados.

In [ ]:
input_path = "./data/crops_animals/crops_animals.csv"

cols = ['Area Code','Area Code (M49)','Area','Item Code','Item Code (CPC)','Item','Element Code','Element','Year Code','Year','Unit','Value','Flag','Note']
reader = pd.read_csv(input_path,
                     delimiter=',', skipinitialspace=True)

data_area = []
counter_area = 0
map_area = {}

for _,item in reader.iterrows():

    if(map_area.get(item['Area']) is None):
        obj_area = {'id_area':counter_area, 'code':item['Area Code'], 'country':item['Area']}
        map_area[item['Area']] = obj_area['id_area']
        counter_area = counter_area + 1
        data_area.append(obj_area)

input_path = "./fertilizers/fertilizers.csv"
reader = pd.read_csv(input_path, delimiter=',', skipinitialspace=True, encoding='latin1')

for _,item in reader.iterrows():

    if(map_area.get(item['Area']) is None):
        obj_area = {'id_area':counter_area, 'code':item['Area Code'], 'country':item['Area']}
        map_area[item['Area']] = obj_area['id_area']
        counter_area = counter_area + 1
        data_area.append(obj_area)


input_path = "./emission/emission.csv"
reader = pd.read_csv(input_path, delimiter=',', skipinitialspace=True, encoding='latin1')

for _,item in reader.iterrows():

    if(map_area.get(item['Area']) is None):
        obj_area = {'id_area':counter_area, 'code':item['Area Code'], 'country':item['Area']}
        map_area[item['Area']] = obj_area['id_area']
        counter_area = counter_area + 1
        data_area.append(obj_area)


/tmp/ipykernel_14720/1167144110.py:4: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  reader = pd.read_csv(input_path,
/tmp/ipykernel_14720/1167144110.py:20: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  reader = pd.read_csv(input_path, delimiter=',', skipinitialspace=True, encoding='latin1')


Realiza chamada no postgre para inserir as Áreas

In [17]:
Area = Base.classes.Area
try:
    session.bulk_insert_mappings(Area,data_area)
    session.commit()
    data_area = []
except Exception as e:
    print("Erro ao fazer bulk insert mappings dos Area : ", e)
    session.rollback()

Lê os dados da criação de animais a partir do arquivo csv para inserir na tabela Stocks (traducao seria estoque de animais)

In [ ]:
input_path = './data/crops_animals/crops_animals.csv'
cols = ['Area Code','Area Code (M49)','Area','Item Code','Item Code (CPC)','Item','Element Code','Element','Year Code','Year','Unit','Value','Flag','Note']
reader = pd.read_csv(input_path, 
                    delimiter=',')

data_stocks = []
counter_stocks = 0
map_stocks = {}

for _,item in reader.iterrows():

    if item['Element Code'] == 5111 or item['Element Code'] == 5112 or item['Element Code'] == 5114:
        counter_stocks = counter_stocks + 1
        obj_stocks = {'id_stocks':counter_stocks, 'year':item['Year'], 'unit':item['Unit'], 'value':item['Value'], 'id_animals':map_animals[item['Item']], 'id_area':map_area[item['Area']]}
        
        if(obj_stocks['unit'] == '1000 An' or obj_stocks['unit'] == '1000 No'):
            obj_stocks['value'] *= 1000
            if(obj_stocks['unit'] == '1000 An'):
                obj_stocks['unit'] = 'An'
            if(obj_stocks['unit'] == '1000 No'):
                obj_stocks['unit'] = 'No'

        data_stocks.append(obj_stocks)
        


/tmp/ipykernel_14720/951022557.py:3: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  reader = pd.read_csv(input_path,


Realiza chamada no banco para inserir os Stocks, há esse linhas de DELETE porque havia dados de todos os animais juntos: Meat, Total que é um dado derivado, contra a 3 Forma Normalizada, e também traria erros as consultas. E também retiramos eventuais dados que não foram preenchidos com NULL no csv.

In [ ]:
try:
    session.bulk_insert_mappings(Stocks,data_stocks)
    session.commit()
    data_stocks = []
    session.execute(text('DELETE FROM "Stocks" WHERE id_animals=21'))    
    session.commit()
    session.execute(text('DELETE FROM "Animals" WHERE name="Meat, Total"'))
    session.commit()
    session.execute(text('DELETE FROM "Stocks" WHERE value="NaN"'))
    session.commit()

except Exception as e:
    print("Erro ao fazer bulk insert mappings dos Stocks : ", e)
    session.rollback()

Erro ao fazer bulk insert mappings dos Stocks :  Textual SQL expression 'DELETE FROM "Stocks" WHER...' should be explicitly declared as text('DELETE FROM "Stocks" WHER...')


Lê os dados da producao de alimentos a partir do arquivo csv para inserir na tabela Production

In [ ]:
input_path = './data/crops_animals/crops_animals.csv'
cols = ['Area Code','Area Code (M49)','Area','Item Code','Item Code (CPC)','Item','Element Code','Element','Year Code','Year','Unit','Value','Flag','Note']
reader = pd.read_csv(input_path, 
                    delimiter=',')

data_production = []
counter_production = 0
map_production = {}

for _,item in reader.iterrows():

    if item['Element Code'] == 5322 or item['Element Code'] == 5323 or item['Element Code'] == 5510 or item['Element Code'] == 5513:
        counter_production = counter_production + 1
        obj_production = {'id_production':counter_production, 'year':item['Year'], 'unit':item['Unit'], 'value':item['Value'], 'id_food':map[item['Item']], 'id_area':map_area[item['Area']]}
        if item['Unit'] == None or  isinstance(item['Unit'], numbers.Number):
            obj_production['unit'] = 0
        if item['Unit'] == "1000 No":
            obj_production['value'] *= 0.1
            item['Unit'] = 't'
        data_production.append(obj_production)
        


/tmp/ipykernel_14720/468031748.py:3: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  reader = pd.read_csv(input_path,


Realiza chamada para inserir producation no postgre

In [ ]:
try:
    session.bulk_insert_mappings(Production,data_production)
    session.commit()
    session.execute(text('DELETE FROM "Production" WHERE value="NaN"'))
    session.commit()
    data_production = []
except Exception as e:
    print("Erro ao fazer bulk insert mappings dos Production : ", e)
    session.rollback()


Lê os dados da area utilizada do arquivo csv para inserir na tabela Area_harvest 

In [ ]:
input_path = './data/crops_animals/crops_animals.csv'
cols = ['Area Code','Area Code (M49)','Area','Item Code','Item Code (CPC)','Item','Element Code','Element','Year Code','Year','Unit','Value','Flag','Note']
reader = pd.read_csv(input_path, 
                    delimiter=',')

data_area_harvested = []
counter_area_harvested = 0
map_area_harvested = {}

for _,item in reader.iterrows():

    if item['Element Code'] == 5312:
        counter_area_harvested = counter_area_harvested + 1
        obj_area_harvest = {'id_area_harvest':counter_area_harvested, 'year':item['Year'], 'scale':item['Unit'], 'value':item['Value'], 'id_food':map[item['Item']], 'id_area':map_area[item['Area']]}
        data_area_harvested.append(obj_area_harvest)
        


/tmp/ipykernel_14720/3859528264.py:3: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  reader = pd.read_csv(input_path,


Realiza chamada para inserir no postgre

In [23]:
try:
    session.bulk_insert_mappings(Area_harvest,data_area_harvested)
    session.commit()
    data_area_harvested = []
except Exception as e:
    print("Erro ao fazer bulk insert mappings dos Area Harvest : ", e)
    session.rollback()


Lê os dados dos gases do arquivo csv para inserir na tabela Gases

In [ ]:
input_path = "./data/emission/emission_elements.csv"
reader = pd.read_csv(input_path,
                     delimiter=',', skipinitialspace=True)

data_Gases = []
counter_Gases = 0
map_Gases_id = {}
for _,item in reader.iterrows():

    if(item['Element Code'] not in map_Gases_id):
        obj_area = {'id_gases':counter_Gases, 'code':item['Element Code'], 'name':item['Element']}
        map_Gases_id[item['Element Code']] = counter_Gases
        counter_Gases = counter_Gases + 1
        data_Gases.append(obj_area)




Realiza chamada no postgre para inserir gases

In [27]:
Gases = Base.classes.Gases
try:
    session.bulk_insert_mappings(Gases,data_Gases)
    session.commit()
    data_Gases = []
except Exception as e:
    print("Erro ao fazer bulk insert mappings dos Gases : ", e)
    session.rollback()

Lê os dados dos Residuos do arquivo csv para inserir na tabela Residue 

In [ ]:
input_path = "./data/emission/emission.csv"

reader = pd.read_csv(input_path,
                     delimiter=',', skipinitialspace=True)

data_Residue = []
counter_Residue = 0
map_Residue_id = {}
for _,item in reader.iterrows():

    if(item['Item Code'] not in map_Residue_id):
        obj_area = {'id_residue':counter_Residue, 'code':item['Item Code'], 'name':item['Item']}
        map_Residue_id[item['Item Code']] = counter_Residue
        counter_Residue = counter_Residue + 1
        data_Residue.append(obj_area)




Realiza chamada no postgre para inserir os residuso

In [29]:
Residue = Base.classes.Residue
try:
    session.bulk_insert_mappings(Residue,data_Residue)
    session.commit()
    data_Residue = []
except Exception as e:
    print("Erro ao fazer bulk insert mappings dos Residue : ", e)
    session.rollback()

Lê os dados dos Fertilizantes do arquivo csv para inserir na tabela Fertilizers

In [ ]:
input_path = "./data/fertilizers/fertilizers.csv"
reader = pd.read_csv(input_path,
                     delimiter=',', skipinitialspace=True, encoding= 'latin1')

data_fertilizer = []
counter_fertilizer = 0
map_fertilizer_id = {}
for _,item in reader.iterrows():

    if(item['Item Code'] not in map_fertilizer_id):
        obj_area = {'id_fertilizer':counter_fertilizer, 'code':item['Item Code'], 'name':item['Item']}
        map_fertilizer_id[item['Item Code']] = counter_fertilizer
        counter_fertilizer = counter_fertilizer + 1
        data_fertilizer.append(obj_area)




/tmp/ipykernel_14720/165871410.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  reader = pd.read_csv(input_path,


Realiza chamada no postgre para inserir fertilizantes

In [31]:
Fertilizer = Base.classes.Fertilizer
try:
    session.bulk_insert_mappings(Fertilizer,data_fertilizer)
    session.commit()
    data_fertilizer = []
except Exception as e:
    print("Erro ao fazer bulk insert mappings dos Fertilizer : ", e)
    session.rollback()

Lê os dados das Emissoes utilizada do arquivo csv para inserir na tabela Emissions 

In [ ]:
input_path = "./data/emission/emission.csv"
reader = pd.read_csv(input_path,
                     delimiter=',', skipinitialspace=True)

data_emission = []
counter_emission = 0

for _, item in reader.iterrows():
    data_emission.append({'id_emissions':counter_emission, 'year':item['Year'], 'unit':item['Unit'], 'value':item['Value'], 'id_residue':map_Residue_id[item['Item Code']],'id_gases':map_Gases_id[item['Element Code']], 'id_area': map_area[item['Area']]})
    counter_emission = counter_emission + 1

Realiza chamada no postgre para inserir as Emissoes

In [33]:
Emissions = Base.classes.Emissions
try:
    session.bulk_insert_mappings(Emissions,data_emission)
    session.commit()
    data_emission = []
except Exception as e:
    print("Erro ao fazer bulk insert mappings dos Emissions : ", e)
    session.rollback()

Insere Fertilizer_production

In [ ]:
input_path = "./data/fertilizers/fertilizers.csv"
reader = pd.read_csv(input_path,
                     delimiter=',', skipinitialspace=True, encoding='latin1')

data_fertilizer_production = []
counter_fertilizer_production = 0

for _,item in reader.iterrows():
    if(item['Element Code'] == 5510):

        data_fertilizer_production.append({'id_fertilizer_production' : counter_fertilizer_production, 'year':item['Year'], 'unit':item['Unit'], 'value':item['Value'], 'id_fertilizer':map_fertilizer_id[item['Item Code']], 'id_area':map_area[item['Area']]})
        counter_fertilizer_production = counter_fertilizer_production + 1


/tmp/ipykernel_14720/1227764859.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  reader = pd.read_csv(input_path,


In [35]:
Fertilizer_production = Base.classes.Fertilizer_production
try:
    session.bulk_insert_mappings(Fertilizer_production, data_fertilizer_production)
    session.commit()
    data_fertilizer_production = []
except Exception as e:
    print("Erro ao fazer bulk insert mappings dos Fertilizer : ", e)
    session.rollback()

Insere Use_per_Area

In [ ]:
input_path = "./data/fertilizers/fertilizers.csv"
reader = pd.read_csv(input_path,
                     delimiter=',', skipinitialspace=True, encoding='latin1')

data_use_per_area = []
counter_use_per_area = 0

for _,item in reader.iterrows():
    if(item['Element Code'] == 5159):

        data_use_per_area.append({'id_use_per_area' : counter_use_per_area, 'year':item['Year'], 'unit':item['Unit'], 'value':item['Value'], 'id_fertilizer':map_fertilizer_id[item['Item Code']], 'id_area':map_area[item['Area']]})
        counter_use_per_area = counter_use_per_area + 1


/tmp/ipykernel_14720/4179709664.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  reader = pd.read_csv(input_path,


In [37]:
Use_per_area = Base.classes.Use_per_area
try:
    session.bulk_insert_mappings(Use_per_area, data_use_per_area)
    session.commit()
    data_use_per_area = []
except Exception as e:
    print("Erro ao fazer bulk insert mappings dos Use_per_area : ", e)
    session.rollback()

Insere Agriculture_use

In [ ]:
input_path = "./data/fertilizers/fertilizers.csv"
reader = pd.read_csv(input_path,
                     delimiter=',', skipinitialspace=True, encoding='latin1')

data_agriculture_use = []
counter_agriculture_use = 0

for _,item in reader.iterrows():
    if(item['Element Code'] == 5157):
        if(item['Year'] == 't' or map_fertilizer_id[item['Item Code']] == 't' or map_area[item['Area']] == 't'):
            print(counter_agriculture_use)
        data_agriculture_use.append({'id_agriculture_use' : counter_agriculture_use, 'year':item['Year'], 'unit':item['Unit'], 'value':item['Value'], 'id_fertilizer':map_fertilizer_id[item['Item Code']], 'id_area':map_area[item['Area']]})
        counter_agriculture_use = counter_agriculture_use + 1


/tmp/ipykernel_14720/2953839290.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  reader = pd.read_csv(input_path,


In [39]:
Agriculture_use = Base.classes.Agriculture_use
try:
    session.bulk_insert_mappings(Agriculture_use, data_agriculture_use)
    session.commit()
    data_agriculture_use = []
except Exception as e:
    print("Erro ao fazer bulk insert mappings dos Agriculture_use : ", e)
    session.rollback()